In [1]:
import xgboost as xgb
from sklearn.datasets import load_svmlight_file

#  This script demonstrate how to do ranking with XGBRanker
x_train, y_train = load_svmlight_file("mq2008.train")
x_valid, y_valid = load_svmlight_file("mq2008.vali")
x_test, y_test = load_svmlight_file("mq2008.test")

display(x_train, y_train, x_valid, y_valid, x_test, y_test)

group_train = []
with open("mq2008.train.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_train.append(int(line.split("\n")[0]))

group_valid = []
with open("mq2008.vali.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_valid.append(int(line.split("\n")[0]))

params = {'objective': 'rank:ndcg', 'learning_rate': 0.1,
          'gamma': 1.0, 'min_child_weight': 0.1,
          'max_depth': 6, 'n_estimators': 4}

model = xgb.sklearn.XGBRanker(**params)
model.fit(x_train, y_train, group_train, verbose=True,
          eval_set=[(x_valid, y_valid)], eval_group=[group_valid])

pred = model.predict(x_test)

<9630x46 sparse matrix of type '<class 'numpy.float64'>'
	with 233775 stored elements in Compressed Sparse Row format>

array([0., 0., 0., ..., 0., 0., 0.])

<2707x46 sparse matrix of type '<class 'numpy.float64'>'
	with 64764 stored elements in Compressed Sparse Row format>

array([0., 0., 1., ..., 0., 0., 0.])

<2874x46 sparse matrix of type '<class 'numpy.float64'>'
	with 71241 stored elements in Compressed Sparse Row format>

array([0., 0., 0., ..., 1., 0., 0.])

[0]	eval_0-map:0.70906
[1]	eval_0-map:0.72783
[2]	eval_0-map:0.72909
[3]	eval_0-map:0.73380


In [4]:
import pandas as pd
import xgboost as xgb
from sklearn.datasets import load_svmlight_file

testdf = pd.read_pickle("../pickles/df_small_clean_test.pkl")
traindf = pd.read_pickle("../pickles/df_small_clean.pkl")

print(testdf.columns)
print(traindf.columns)

# split label and other variables
x_train, y_train = traindf.drop(columns=["position"]), traindf["position"]
x_test, y_test = testdf.drop(columns=["position", "click_bool", "booking_bool"]), testdf["position"]

# params = {'objective': 'rank:ndcg', 'learning_rate': 0.1,
#           'gamma': 1.0, 'min_child_weight': 0.1,
#           'max_depth': 6, 'n_estimators': 4}


# model = xgb.sklearn.XGBRanker(**params)
# model.fit(x_train, y_train, group_train, verbose=True)
# pred = model.predict(x_test)

Index(['prop_id', 'srch_id', 'position', 'price_quality', 'competitor_bool',
       'availability_bools', 'visited_before', 'click_bool', 'booking_bool',
       'PriceBand', 'promotion_flag', 'prop_brand_bool', 'random_bool'],
      dtype='object')
Index(['prop_id', 'srch_id', 'position', 'price_quality', 'competitor_bool',
       'availability_bools', 'visited_before', 'click_bool', 'booking_bool',
       'PriceBand', 'promotion_flag', 'prop_brand_bool', 'random_bool'],
      dtype='object')
